In [3]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient()
db = client.openfoodfacts
collection = db.products

In [4]:
proj = { 
    'product_name_fr' : 1, 
    'ingredients' : 1, 
    'salt_content' : 1, 
    'oil_content' : 1, 
    'product_name' : 1,
    'nutriments' : 1,
    'compared_to_category' : 1,
    'interface_version_modified' : 1
}

addFields = { 
    
    'ingredients' : "$ingredients.text",
    
    'salt_content' : {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ 
                   { '$eq' : [ "$$item.text", 'Sel' ] }, 
                   { '$eq' : [ "$$item.text", 'sel' ] },
                   { '$eq' : [ "$$item.text", 'SEL' ] },
                   { '$eq' : [ "$$item.text", 'VINAGE SEL' ] },
                   { '$eq' : [ "$$item.text", 'sel marin' ] },
                   { '$eq' : [ "$$item.text", 'Sel marin' ] },
                   { '$eq' : [ "$$item.text", 'salt' ] },
                   { '$eq' : [ "$$item.text", 'Salt' ] }] }
            }
    },
    
    
    'oil_content': {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile extra vierge d\'olive' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra-vierge' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile dolive* vierge exta' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'huile de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extra extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'huile de tournesol désodorisée' ] }, {
                   '$eq' : [ "$$item.text", 'Extra Virgin Olive Oil' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile de palme' ] }, {
                   '$eq' : [ "$$item.text", 'natives Olivenöl extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Sonnenblumenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Olivenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile Végétale' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de colza' ] }, {
                   '$eq' : [ "$$item.text", 'huile végétale de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile végétale de colza' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] } ] 
                    }
            }
    },
}




match = {
    
    'ingredients' : { 
        "$elemMatch" : { 
            "id" : { '$regex' : "tomate*", '$options' : "i" },
        } 
    },
    
    "product_name_fr" : { '$regex' : "sauce*", '$options' : "i" }
    
}



c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    { '$addFields' : addFields }
])

data = pd.DataFrame(list(c_f))

In [5]:
data.head(5)

,_id,nutriments,product_name,interface_version_modified,compared_to_category,ingredients,product_name_fr,salt_content,oil_content
0,0000069162176,"{'salt_unit': '', 'energy': 586, 'proteins': 4...",Quenelles de brochet sauce Nantua,20150316.jqm2,en:pike-quenelles,"[INGREDIENTES, agua, harina de tngo tierno car...",Quenelles de brochet sauce Nantua,[],[]
1,00005010,"{'energy-kcal_value': 106, 'sodium_100g': 0.16...",Organic tomato sauce bolognese,20150316.jqm2,en:bolognese-sauces,"[Tomato puree, beef mince meat, carrots, extra...",Organic tomato sauce bolognese,"[{'id': 'en:salt', 'text': 'salt', 'vegetarian...",[]
2,00021036,"{'salt': 0.6, 'fiber_100g': 3.1, 'carbohydrate...",Tomato & Gorgonzola pasta sauce,20150316.jqm2,NaN,"[sauce à la tomate, à Itoignon kJ et à la caro...",Tomato & Gorgonzola pasta sauce,[],[]
3,00089227,"{'fiber': 1.7, 'fruits-vegetables-nuts-estimat...",Tomato & Sausage Pasta Sauce,20120622,en:tomato-sauces,"[Pulpe de tomate, Saucisse de porc, Concentré ...",Tomato & Sausage Pasta Sauce,"[{'id': 'en:salt', 'vegan': 'yes', 'text': 'Se...","[{'from_palm_oil': 'no', 'id': 'en:extra-virgi..."
4,0011110791474,"{'sodium_100g': 2, 'energy-kcal_value': 100, '...",Peri peri chili culinary hot sauce,20150316.jqm2,en:sauces,"[Water, red bell peppers, distilled vinegar, t...",Peri peri chili culinary hot sauce,"[{'percent_min': 0, 'rank': 6, 'percent_max': ...",[]


In [9]:
max_salt_content = []
for l in data['salt_content'] :
    if len(l) > 0 and 'percent_max' in l[0]  :
        max_salt_content.append(l[0]['percent_max'])
    else :
        max_salt_content.append(0)
        
data['max_salt_content'] = max_salt_content

min_salt_content = []
for l in data['salt_content'] :
    if len(l) > 0 and 'percent_min' in l[0]  :
        min_salt_content.append(l[0]['percent_min'])
    else :
        min_salt_content.append(0)
        
data['min_salt_content'] = min_salt_content

max_oil_content = []
for l in data['oil_content'] :
    if len(l) > 0 and 'percent_max' in l[0]  :
        max_oil_content.append(l[0]['percent_max'])
    else :
        max_oil_content.append(0)
        
data['max_oil_content'] = max_oil_content

min_oil_content = []
for l in data['oil_content'] :
    if len(l) > 0 and 'percent_min' in l[0]  :
        min_oil_content.append(l[0]['percent_min'])
    else :
        min_oil_content.append(0)
        
data['min_oil_content'] = min_oil_content

oil_type = []
for l in data['oil_content'] :
    if len(l) > 0 and 'percent_min' in l[0]  :
        oil_type.append(l[0]['text'])
    else :
        oil_type.append("")
        
data['oil_type'] = oil_type

salt_100g = []
for l in data['nutriments'] :
    if len(l) > 0 and 'salt_100g' in l :
        salt_100g.append(l['salt_100g'])
    else :
        salt_100g.append(None)
        
data['salt_100g'] = salt_100g

fiber_100g = []
for l in data['nutriments'] :
    if len(l) > 0 and 'fiber_100g' in l :
        fiber_100g.append(l['fiber_100g'])
    else :
        fiber_100g.append(None)
        
data['fiber_100g'] = fiber_100g

sodium_100g = []
for l in data['nutriments'] :
    if len(l) > 0 and 'sodium_100g' in l :
        sodium_100g.append(l['sodium_100g'])
    else :
        sodium_100g.append(None)
        
data['sodium_100g'] = sodium_100g

sugars_100g = []
for l in data['nutriments'] :
    if len(l) > 0 and 'sugars_100g' in l :
        sugars_100g.append(l['sugars_100g'])
    else :
        sugars_100g.append(None)
        
data['sugars_100g'] = sugars_100g


saturated_fat_100g = []
for l in data['nutriments'] :
    if len(l) > 0 and 'saturated-fat_100g' in l :
        saturated_fat_100g.append(l['saturated-fat_100g'])
    else :
        saturated_fat_100g.append(None)
        
data['saturated_fat_100g'] = saturated_fat_100g


nutrition_score_fr_100g = []
for l in data['nutriments'] :
    if len(l) > 0 and 'nutrition-score-fr_100g' in l :
        nutrition_score_fr_100g.append(l['nutrition-score-fr_100g'])
    else :
        nutrition_score_fr_100g.append(None)
        
data['nutrition_score_fr_100g'] = nutrition_score_fr_100g


energy_100g = []
for l in data['nutriments'] :
    if len(l) > 0 and 'energy_100g' in l :
        energy_100g.append(l['energy_100g'])
    else :
        energy_100g.append(None)
        
data['energy_100g'] = energy_100g





In [10]:
data

,_id,nutriments,product_name,interface_version_modified,compared_to_category,ingredients,product_name_fr,salt_content,oil_content,max_salt_content,...,max_oil_content,min_oil_content,oil_type,salt_100g,fiber_100g,sodium_100g,sugars_100g,saturated_fat_100g,nutrition_score_fr_100g,energy_100g
0,0000069162176,"{'salt_unit': '', 'energy': 586, 'proteins': 4...",Quenelles de brochet sauce Nantua,20150316.jqm2,en:pike-quenelles,"[INGREDIENTES, agua, harina de tngo tierno car...",Quenelles de brochet sauce Nantua,[],[],0,...,0,0,,4.9000,0.5,1.96000,0.6,5.50,16.0,586.0
1,00005010,"{'energy-kcal_value': 106, 'sodium_100g': 0.16...",Organic tomato sauce bolognese,20150316.jqm2,en:bolognese-sauces,"[Tomato puree, beef mince meat, carrots, extra...",Organic tomato sauce bolognese,"[{'id': 'en:salt', 'text': 'salt', 'vegetarian...",[],1.02307,...,0,0,,0.4000,NaN,0.16000,3.2,2.00,-1.0,444.0
2,00021036,"{'salt': 0.6, 'fiber_100g': 3.1, 'carbohydrate...",Tomato & Gorgonzola pasta sauce,20150316.jqm2,NaN,"[sauce à la tomate, à Itoignon kJ et à la caro...",Tomato & Gorgonzola pasta sauce,[],[],0,...,0,0,,0.6000,3.1,0.24000,3.5,3.00,NaN,410.0
3,00089227,"{'fiber': 1.7, 'fruits-vegetables-nuts-estimat...",Tomato & Sausage Pasta Sauce,20120622,en:tomato-sauces,"[Pulpe de tomate, Saucisse de porc, Concentré ...",Tomato & Sausage Pasta Sauce,"[{'id': 'en:salt', 'vegan': 'yes', 'text': 'Se...","[{'from_palm_oil': 'no', 'id': 'en:extra-virgi...",0,...,0,0,,0.7900,1.7,0.31600,4.6,2.50,3.0,556.0
4,0011110791474,"{'sodium_100g': 2, 'energy-kcal_value': 100, '...",Peri peri chili culinary hot sauce,20150316.jqm2,en:sauces,"[Water, red bell peppers, distilled vinegar, t...",Peri peri chili culinary hot sauce,"[{'percent_min': 0, 'rank': 6, 'percent_max': ...",[],16.6667,...,0,0,,5.0000,NaN,2.00000,0.0,NaN,NaN,418.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,93201834,"{'energy-kcal_unit': 'kcal', 'sugars': 46, 'pr...",BBQ Sauce,20150316.jqm2,en:barbecue-sauces,"[sugar, water, white vinegar, tomato paste mal...",BBQ Sauce,"[{'vegetarian': 'yes', 'percent_max': 16.66666...",[],16.6667,...,0,0,,1.1000,NaN,0.44000,46.0,1.00,24.0,3556.0
2704,9300681008403,"{'fat_unit': 'g', 'nova-group_serving': 4, 'sa...",Sweet chili sauce,20150316.jqm2,fr:sauces-tomates-pimentees,"[Sucre, eau, concentré de tomates, vinaigre, l...",Sweet chili sauce,"[{'text': 'sel', 'id': 'en:salt', 'percent_min...",[],2.5,...,0,0,,2.6162,NaN,1.04648,42.7,0.02,21.0,804.0
2705,9556041601214,"{'carbohydrates_unit': '', 'saturated-fat_100g...",Ayam Sardines In Tomato Sauce,20150316.jqm2,en:sardines-in-tomato-sauce,"[POISSON DE TYPE SARDINES, SAUCE TOMATE, PILCH...",Ayam Sardines In Tomato Sauce,"[{'vegan': 'yes', 'vegetarian': 'yes', 'text':...",[],0,...,0,0,,0.9200,NaN,0.36800,1.1,2.70,2.0,565.0
2706,9556041611121,"{'sodium': 1.08, 'proteins_serving': 0, 'salt'...",Sauce aigre-douce Ayam™,20190830,en:sweet-and-sour-sauces,"[EAU, SUCRE, ÉPAISSISSANT, CONCENTRÉ DE TOMATE...",Sauce aigre-douce Ayam™,"[{'percent_min': 0, 'vegetarian': 'yes', 'vega...",[],16.6667,...,0,0,,2.7000,NaN,1.08000,46.0,0.30,22.0,795.0


In [28]:
data.to_csv('O_ingredients.csv')